In [1]:
import azureml.core
from azureml.core import Workspace, Model

# Inference testing function definition

In [2]:
import base64
import requests
import json 

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def test_inf(endpoint, key=None):
    input_json = json.dumps({
        "n_cards": 5,
        "images": True,
        "hand": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/test/IMG_4903.jpg'),
        "deck": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/train/A_1.jpg')
    })
    headers = { 'Content-Type':'application/json' }
    if key:
        headers['Authorization'] = f'Bearer {key}' # Only difference to local is to send the auth key
    predictions = requests.post(endpoint, input_json, headers = headers)
    p = predictions.json()
    print(str(p)+'\n')
    print(f'Hand:     {p[0]}')
    print(f'Deck:     {p[1]}')
    if p[2]:
        play = p[2][0]
        print(f'Score:    {play[0]}')
        print(f'Complete: {play[1]}')
        print(f'Words:    {[c[0] for c in play[2]]}')
        print(f'Pick up:  {p[2][1]}')
        print(f'Drop:     {p[2][2]}\n')
    else:
        print('No possible play for these cards')    

# Connect to the Workspace

In [3]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print(f'Azure ML version: {azureml.core.VERSION}, Workspace: {ws.name}')

Azure ML version: 1.20.0, Workspace: jbWorkspace


# Create the inference environment

In [4]:
from azureml.core.webservice import LocalWebservice, AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [11]:
myenv = Environment(name="quiddler-inference-env")
myenv.inferencing_stack_version='latest'
myenv.docker.enabled = True
myenv.docker.base_image = None

# Specify docker steps as a string.
# see https://github.com/Azure/AzureML-Containers
# Needed to add libgl1-mesa-dev to resolve:
#  ImportError: libGL.so.1: cannot open shared object file: No such file or directory
dockerfile = r'''
FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
RUN apt-get install -y libgl1-mesa-dev
'''
myenv.docker.base_dockerfile = dockerfile

conda_dep = CondaDependencies(conda_dependencies_file_path='inference-env.yml')
myenv.python.conda_dependencies=conda_dep

In [9]:
model = ws.models['quiddler_model']
print(f'Model: {model.name}, Version: {model.version}')

Model: quiddler_model, Version: 1


# Deploy locally

In [12]:
# Configure the scoring environment
inference_config = InferenceConfig(source_directory = 'source',
                                   entry_script="score.py",
                                   environment=myenv)

deployment_config = LocalWebservice.deploy_configuration()

service_name = "quiddler-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Generating Docker build context.
2021/01/19 20:56:43 Downloading source code...
2021/01/19 20:56:45 Finished downloading source code
2021/01/19 20:56:46 Creating Docker network: acb_default_network, driver: 'bridge'
2021/01/19 20:56:46 Successfully set up Docker network: acb_default_network
2021/01/19 20:56:46 Setting up Docker configuration...
2021/01/19 20:56:47 Successfully set up Docker configuration
2021/01/19 20:56:47 Logging in to registry: cb7e3186329f4aacb35a6ed2e3e03cb4.azurecr.io
2021/01/19 20:56:48 Successfully logged into cb7e3186329f4aacb35a6ed2e3e03cb4.azurecr.io
2021/01/19 20:56:48 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/01/19 20:56:48 Scanning for dependencies...
2021/01/19 20:56:50 Successfully scanned dependencies
2021/01/19 20:56:50 Launching container with name: acb_step_0
Sending build context to Docker daemon  65.02kB
Step 1/23 : FROM mcr.microsoft.com/azureml/o16n-base/python-assets:latest AS 

 ---> Running in 3e7efdf84f70
Removing intermediate container 3e7efdf84f70
 ---> a8efbfe28cfe
Step 5/23 : RUN mkdir -p $HOME/.cache
 ---> Running in 3702466fda67
Removing intermediate container 3702466fda67
 ---> 24043366fe99
Step 6/23 : WORKDIR /
 ---> Running in d50909bdc5cf
Removing intermediate container d50909bdc5cf
 ---> bd1656818124
Step 7/23 : COPY azureml-environment-setup/99brokenproxy /etc/apt/apt.conf.d/
 ---> 2004f3070a66
Step 8/23 : RUN true
 ---> Running in a8d9370fcfc3
Removing intermediate container a8d9370fcfc3
 ---> ba97f18d6825
Step 9/23 : COPY --from=inferencing-assets /artifacts /var/
 ---> 82376efc00a0
Step 10/23 : RUN /var/requirements/install_system_requirements.sh
 ---> Running in fc8af322fb07
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://packages.microsoft.com/ubuntu/18.04/prod bionic InRelease [4003 B]
Hit:3 http://ppa.launchpad.net/adiscon/v8-stable/ubuntu bionic InRelease
Get:4 https://packages.microsoft.com/ubun

done
Solving environment: ...working... done

libedit-3.1.20191231 | 121 KB    | ########## | 100% 
tk-8.6.10            | 3.2 MB    | ########## | 100% 
sqlite-3.33.0        | 2.0 MB    | ########## | 100% 
zlib-1.2.11          | 120 KB    | ########## | 100% 
ld_impl_linux-64-2.3 | 645 KB    | ########## | 100% 
ncurses-6.2          | 1.1 MB    | ########## | 100% 
ca-certificates-2020 | 128 KB    | ########## | 100% 
readline-8.0         | 428 KB    | ########## | 100% 
xz-5.2.5             | 438 KB    | ########## | 100% 
libgcc-ng-9.1.0      | 8.1 MB    | ########## | 100% 
wheel-0.35.1         | 36 KB     | ########## | 100% 
openssl-1.1.1h       | 3.8 MB    | ########## | 100% 
libstdcxx-ng-9.1.0   | 4.0 MB    | ########## | 100% 
python-3.8.5         | 57.4 MB   | ########## | 100% 
pip-20.2.4           | 2.0 MB    | ########## | 100% 
setuptools-50.3.0    | 902 KB    | ########## | 100% 
libffi-3.3           | 54 KB     | ########## | 100% 
certifi-2020.6.20    | 160 KB    | #

Removing intermediate container e496e02a48b3
 ---> 9a5b6e87d8ad
Step 16/23 : ENV PATH /azureml-envs/azureml_adf7e3daaad3fb8c95ce06e75772864e/bin:$PATH
 ---> Running in 06ae25521f2c
Removing intermediate container 06ae25521f2c
 ---> 6490852b53cc
Step 17/23 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_adf7e3daaad3fb8c95ce06e75772864e
 ---> Running in 4a552d59fe6f
Removing intermediate container 4a552d59fe6f
 ---> 376298a452be
Step 18/23 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_adf7e3daaad3fb8c95ce06e75772864e/lib:$LD_LIBRARY_PATH
 ---> Running in 75b48092e2ef
Removing intermediate container 75b48092e2ef
 ---> 0cb2d5acdc84
Step 19/23 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> e6d98dc6841d
Step 20/23 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit  /azureml-environment-setup/spark_cache.py'; fi
 ---> Running in 0b247db0dc65
Removing intermediate container 0b247db0dc6

## Call the inference service

In [ ]:
# Reload the service to pick up changes in the source directory
service.reload(wait=True)

In [13]:
local_endpoint = service.scoring_uri
print(local_endpoint)

http://localhost:49164/score


In [14]:
test_inf(local_endpoint)

['g/th/in/p/u', 'a', [[32, True, [['g/u/p', [16, 3, ['g', 'u', 'p']]], ['th/in', [16, 2, ['th', 'in']]]]], 'a', 'a']]

Hand:     g/th/in/p/u
Deck:     a
Score:    32
Complete: True
Words:    ['g/u/p', 'th/in']
Pick up:  a
Drop:     a



In [15]:
print(service.get_logs())

2021-01-19T21:10:59,326719140+00:00 - iot-server/run 
2021-01-19T21:10:59,326881937+00:00 - rsyslog/run 
2021-01-19T21:10:59,326883285+00:00 - gunicorn/run 
2021-01-19T21:10:59,329049501+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-01-19T21:10:59,386905094+00:00 - iot-server/finish 1 0
2021-01-19T21:10:59,387707865+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 49
/azureml-envs/azureml_adf7e3daaad3fb8c95ce06e75772864e/lib/python3.8/os.py:1023: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  return io.open(fd, *args, **kwargs)
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6

## Shut it down

In [16]:
service.delete()
print("Service deleted.")

Container has been successfully cleaned up.
Service deleted.


In [ ]:
!docker container ls

# Deploy to Azure Container Instance

In [ ]:
# ACI Service
# Configure the scoring environment
inference_config = InferenceConfig(source_directory = 'source',
                                   entry_script="score.py",
                                   environment=myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights=True)

service_name = "quiddler-service"

aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
aci_endpoint = aci_service.scoring_uri
print(aci_endpoint)

In [ ]:
test_inf(aci_endpoint)

In [ ]:
aci_service.delete()

# Deploy into AKS

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

service_name = "quiddler-service2"
aks_target = AksCompute(ws,"jb-inf")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aks_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)
#print(aks_service.get_logs())

In [ ]:
primary, secondary = aks_service.get_keys()
aks_endpoint = aks_service.scoring_uri
print(aks_endpoint)
print(primary)

In [ ]:
test_inf(aks_endpoint, primary)

In [ ]:
aks_service.delete()